In [1]:
!pip install scikit-learn
import pickle, torch, torch.nn as nn, numpy as np
import sklearn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ------------------------
# Load windows
# ------------------------
train = pickle.load(open("./windows/train_windows.pkl","rb"))
test  = pickle.load(open("./windows/test_windows.pkl","rb"))
companies = pickle.load(open("./windows/company_list.pkl","rb"))

# Encode companies
enc = LabelEncoder().fit(companies)
train = [(X, y, enc.transform([t])[0]) for X, y, t in train]
test  = [(X, y, enc.transform([t])[0]) for X, y, t in test]

# ------------------------
# Normalize features
# ------------------------
all_X = np.concatenate([X for X, _, _ in train], axis=0)
mean, std = all_X.mean(axis=0), all_X.std(axis=0) + 1e-6

def normalize_windows(data):
    normalized = []
    for X, y, t in data:
        X = (X - mean) / std
        X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
        normalized.append((X, y, t))
    return normalized

train = normalize_windows(train)
test  = normalize_windows(test)

# ------------------------
# Dataset and DataLoader
# ------------------------
class WindowDataset(Dataset):
    def __init__(self, data): self.data=data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        X, y, c = self.data[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32), torch.tensor(c, dtype=torch.long)

train_loader = DataLoader(WindowDataset(train), batch_size=64, shuffle=True)
test_loader  = DataLoader(WindowDataset(test), batch_size=64, shuffle=False)

# ------------------------
# Transformer model
# ------------------------
class StockTransformer(nn.Module):
    def __init__(self, feature_dim, num_companies):
        super().__init__()
        d_model = 128
        self.input_proj = nn.Linear(feature_dim, d_model)
        self.company_emb = nn.Embedding(num_companies, d_model)
        enc_layer = nn.TransformerEncoderLayer(d_model, nhead=4, batch_first=True)
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=2)
        self.head = nn.Sequential(nn.Linear(d_model, 64), nn.ReLU(), nn.Linear(64, 1))
        
    def forward(self, x, c):
        x = self.input_proj(x)
        x = x + self.company_emb(c).unsqueeze(1)
        x = self.encoder(x)
        return self.head(x[:, -1, :]).squeeze(-1)

feature_dim = train[0][0].shape[1]
model = StockTransformer(feature_dim, len(companies)).to(DEVICE)

# ------------------------
# Optimizer & Loss
# ------------------------
opt = torch.optim.Adam(model.parameters(), lr=1e-5)  # lower LR for stability
loss_fn = nn.MSELoss()

# ------------------------
# Training loop
# ------------------------
EPOCHS = 1500
GRAD_CLIP = 1.0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for X, y, c in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        X, y, c = X.to(DEVICE), y.to(DEVICE), c.to(DEVICE)
        opt.zero_grad()
        pred = model(X, c)
        loss = loss_fn(pred, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Train Loss: {total_loss/len(train_loader):.6f}")
    
    # Optional: evaluate on test set every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for X, y, c in test_loader:
                X, y, c = X.to(DEVICE), y.to(DEVICE), c.to(DEVICE)
                pred = model(X, c)
                total_test_loss += loss_fn(pred, y).item()
        print(f"Epoch {epoch+1} Test Loss: {total_test_loss/len(test_loader):.6f}")

# ------------------------
# Save model
# ------------------------
torch.save({"model_state": model.state_dict(), "companies": companies, "mean": mean, "std": std}, "./model_data.pt")
print("✅ Model saved as model_data.pt")

Epoch 1: 100%|██████████| 125/125 [00:01<00:00, 92.47it/s]


Epoch 1 Train Loss: 33577.243969


Epoch 2: 100%|██████████| 125/125 [00:01<00:00, 90.78it/s]


Epoch 2 Train Loss: 33145.018219


Epoch 3: 100%|██████████| 125/125 [00:01<00:00, 90.67it/s]


...

Epoch 1498 Train Loss: 5.041582


Epoch 1499: 100%|██████████| 125/125 [00:01<00:00, 91.01it/s]


Epoch 1499 Train Loss: 5.385649


Epoch 1500: 100%|██████████| 125/125 [00:01<00:00, 90.88it/s]


Epoch 1500 Train Loss: 5.334387
Epoch 1500 Test Loss: 41.868676
✅ Model saved as model.pt


In [13]:
train = pickle.load(open(f"./windows/train_windows.pkl","rb"))
test  = pickle.load(open(f"./windows/test_windows.pkl","rb"))

# Check shapes
print("Train samples:", len(train))
print("Test samples:", len(test))
print("Sample window shape:", train[0][0].shape)
print("Sample target:", train[0][1])

# Check for NaNs or Infs
def check_data(data):
    nan_X = 0
    inf_X = 0
    nan_y = 0
    inf_y = 0
    for X, y, t in data:
        if np.isnan(X).any(): nan_X += 1
        if np.isinf(X).any(): inf_X += 1
        if np.isnan(y): nan_y += 1
        if np.isinf(y): inf_y += 1
    print(f"NaN features: {nan_X}, Inf features: {inf_X}")
    print(f"NaN targets: {nan_y}, Inf targets: {inf_y}")

check_data(train)
check_data(test)

Train samples: 7960
Test samples: 1990
Sample window shape: (8, 778)
Sample target: 46.57400131225586
NaN features: 210, Inf features: 0
NaN targets: 0, Inf targets: 0
NaN features: 47, Inf features: 0
NaN targets: 0, Inf targets: 0
